In [1]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!pip install -q langchain accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00


In [7]:
import torch
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from transformers import BitsAndBytesConfig


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap

In [8]:
model_name = 'bn22/Mistral-7B-Instruct-v0.1-sharded'



In [9]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

In [10]:
 model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model_00001-of-00010.bin:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

pytorch_model_00002-of-00010.bin:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

pytorch_model_00003-of-00010.bin:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

...

pytorch_model_00009-of-00010.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

pytorch_model_00010-of-00010.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

pytorch_model_00011-of-00010.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
mistral_tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [12]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= mistral_tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=mistral_tokenizer.eos_token_id
                )

In [13]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an advanced clinical bot and medical expert.
Always answer as helpfully as possible, while being safe. You use domain knowledge in medical science to give Answers and these answers based on information in the Conversation Transcript which is prompted by the User.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.Just say you don't know and you are sorry!
"""

In [15]:
def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = mistral_tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=mistral_tokenizer.eos_token_id,
                                 pad_token_id=mistral_tokenizer.eos_token_id,
                                 )
        final_outputs = mistral_tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

In [16]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap

In [17]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})

In [18]:
system_prompt = """
You are an advanced clinical bot and medical expert.
Based on the Conversation transcript prompted by the user if this Transcript is relevant to the medical domain use your domain knowledge in medical science and Please consider only information in the Transcript  that might be relevent to each of these sections in the clinical note below  and  if the information regarding  particular section is not present in the Transcript please just respond 'NA' to that particular section in the clinical note below.
"""

instruction = """

Based on the conversation transcript prompted by the user only if this Transcript is relevant to the medical domain use your domain knowledge in medical science and Please consider only information in the Transcript that might be relevent to each of these sections in the clinical note below  and  if the information regarding  particular section is not present in the Transcript please just respond 'NA' to that particular section in the clinical note below.

generate the Clinical Note in this format:

Clinical Note :-

Diagnosis:

History of Presenting Illness:

Medications (Prescribed):

Lab Tests (Ordered):


###

Example only for reference puposes do make up this as an answer :

Conversation Transcript

Patient: I've been taking the Glycomet-GP 1 as you prescribed, doctor, but I'm still feeling quite unwell. My blood pressure readings are all over the place and my sugar levels are high.'

Doctor: I see, we may need to adjust your medications. Let's add Jalra-OD and Telmis to your regimen and see how you respond.

###

Clinical Note :-

Diagnosis: Uncontrolled Diabetes and Hypertension

History of Presenting Illness: The patient has been adhering to their current medication regimen but the diabetes and hypertension seem uncontrolled.

Medications (Prescribed):

Glycomet-GP 1

Jalra-OD

Telmis

Lab Tests (Ordered): None

###

Now, based on the following conversation, please generate a clinical note:

### Conversation Transcript
{Transcript}

"""
template = get_prompt(instruction, system_prompt)
print(template)

[INST]<>

You are an advanced clinical bot and medical expert.
Based on the Conversation transcript prompted by the user if this Transcript is relevant to the medical domain use your domain knowledge in medical science and Please consider only information in the Transcript  that might be relevent to each of these sections in the clinical note below  and  if the information regarding  particular section is not present in the Transcript please just respond 'NA' to that particular section in the clinical note below.

<>



Based on the conversation transcript prompted by the user only if this Transcript is relevant to the medical domain use your domain knowledge in medical science and Please consider only information in the Transcript that might be relevent to each of these sections in the clinical note below  and  if the information regarding  particular section is not present in the Transcript please just respond 'NA' to that particular section in the clinical note below.

generate the 

In [19]:
prompt = PromptTemplate(template=template, input_variables=["Transcript"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)

In [21]:
response = llm_chain.run("What's your interest in it Mr. Wayne? I want to borrow it for spelunking. Spelunking? Yeah, you know, cave diving. You're expecting to run into much gunfire in these caves?")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Clinical Note:

No medical information was present in the conversation transcript. N/A


In [23]:
response = llm_chain.run("Patient's name is Adhiraj Singh Shikhaward. Patient's symptoms are dizziness, cough and cold. I am prescribing medicines to the patient. Paracetamol, Citrazin and Azithromycin.")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Clinical Note :-

Diagnosis: Upper Respiratory Infection

History of Presenting Illness: The patient, Adhiraj Singh Shikhaward, is presenting with symptoms of dizziness, cough, and cold.

Medications (Prescribed):

Paracetamol

Citrazin

Azithromycin

Lab Tests (Ordered): A Chest X-ray and a complete blood count (CBC) to rule out any underlying medical conditions.


In [24]:
response = llm_chain.run(
"""
Patient: I am having dizziness, cough and cold. what should i do?
Doctor: take Paracetamol, Citrazin and Azithromycin.
""")
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Clinical Note :-

Diagnosis: Upper respiratory tract infection

History of Presenting Illness: The patient presents with complaints of dizziness, cough and cold.

Medications (Prescribed):

Paracetamol

Citrazin

Azithromycin

Lab Tests (Ordered): None

NA - There is no laboratory testing mentioned in this conversation transcript.


In [25]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [26]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [27]:
import whisper
import time

In [36]:
whisper_model=whisper.load_model('medium')

100%|█████████████████████████████████████| 1.42G/1.42G [00:16<00:00, 93.3MiB/s]


In [37]:
def add_text(audio):
    time.sleep(3)
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(whisper_model.device)
    _, probs = whisper_model.detect_language(mel)
    options = {"fp16": False, "language": "hi", "task": "translate"}
    result = whisper.decode(whisper_model, mel, **options)
    text = result.text
    #state += text + " "
    return text

In [38]:
add_text('/content/Wednesday at 12-02 PM.m4a')

"Patient's name is Adhiraj Singh Shikhaward. Patient's symptoms are dizziness, cough and cold. I am prescribing medicines to the patient. Paracetamol, Citrazin and Azithromycin."

In [39]:
def generate_response(query, chat_history: list) -> tuple:
    #messages = generate_messages(chat_history, query)
    bot_message = llm_chain.run(query)
    chat_history.append((query, bot_message))
    #time.sleep(random.randint(0, 5))
    return '', chat_history

In [40]:
def user(user_message, history):
        return "", history + [[user_message, None]]

In [48]:
def bot(query,history):
        bot_message=llm_chain.run(query)
        print(bot_message)
        history[-1][1] = ""
        for character in bot_message:
            history[-1][1] += character
            time.sleep(0.05)
            yield history

In [47]:
bot("Patient's name is Adhiraj Singh Shikhaward. Patient's symptoms are dizziness, cough and cold. I am prescribing medicines to the patient. Paracetamol, Citrazin and Azithromycin.",history=[])

<generator object bot at 0x7b31658e9d20>

In [32]:
!pip install -q typing-extensions==4.5.0 ftfy bitsandbytes==0.35.0 gradio natsort

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 739.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.4 M

In [49]:
import gradio as gr

In [50]:
block=gr.Blocks()
with block:
    gr.Markdown("""<h1><center>Chikitsa Bot</center></h1>
    """)
    chatbot = gr.Chatbot()
    with gr.Row():
        message=gr.Textbox()
        with gr.Column():
             btn = gr.components.Audio(source="microphone", type="filepath")
             upload = gr.Audio(sources=["upload"], type="filepath")
             submit = gr.Button("SUBMIT")
    submit.click(add_text, inputs=[btn], outputs=[message]),
    submit.click(add_text,inputs=[upload],outputs=[message])
    message.submit(generate_response, [message, chatbot], [message, chatbot]),


block.queue()
block.launch()

<ipython-input-50-25aca7d65c3b>:10: GradioUnusedKwargWarning: You have unused kwarg parameters in Audio, please remove them: {'sources': ['upload']}
  upload = gr.Audio(sources=["upload"], type="filepath")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://140377220ce5635006.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [51]:
block=gr.Blocks()
with block:
    gr.Markdown("""<h1><center>Chikitsa Bot</center></h1>
    """)
    chatbot = gr.Chatbot()
    with gr.Row():
        message=gr.Textbox()
        with gr.Column():
            btn = gr.components.Audio(source="microphone", type="filepath")
            upload = gr.Audio(sources=["upload"], type="filepath")
            submit = gr.Button("SUBMIT")
    submit.click(add_text, inputs=[btn], outputs=[message]),
    submit.click(add_text,inputs=[upload],outputs=[message])
    message.submit(user, [message, chatbot], [message, chatbot], queue=False).then(
        bot, [message,chatbot],chatbot
    )
block.queue()
block.launch()

<ipython-input-51-3a11c647a617>:10: GradioUnusedKwargWarning: You have unused kwarg parameters in Audio, please remove them: {'sources': ['upload']}
  upload = gr.Audio(sources=["upload"], type="filepath")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://141f9f7cbe2eebdf21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
